# Agrodem

A python tool for estimating water and electricity demand for irrigation. Developed by KTH dESA

## Import modules required

In [3]:
import pandas as pd
from datetime import datetime
import pyeto
import numpy as np
import ast       
import math
from ast import literal_eval  
from pandas import DataFrame
from scipy.interpolate import interp1d
import dateutil     #dateutil module provides powerful extensions to the standard datetime module
from dateutil import parser  #This module offers reads the given date in string and convert it to date format or timestamps,it represent a date and/or time from most known formats 

# note that pyeto is available here https://github.com/woodcrafty/PyETo.git
from pyeto import fao

%matplotlib inline

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

## Part 1 - Biophysical characteristics

### Importing input dataset from GIS 

Main inputs include:

* Country (name)
* Admin 1 (name)
* lat, lon (deg)
* Total cell area (ha)                              
* Harvested area (ha)                                 
* Current yield (kg/ha)                                
* Maximun yield (kg/ha)
* Ground water depth (m)
* Distance to surface water (m)
* Elevation (m)
* Surface water "depth" (m)                            
* Surface irrigation suitability (1,9999)
* Water storage capacity of the soil (mm/m)
* Average wind speed per month (m s-1)
* Min, Average, Max temperature per month (C)
* Average precipitation (mm/month)
* Average solar irradiation per month (kJ m-2 day-1)


This is a sample dataset with about 1000 rows located in Zambezia province in Mozambique. 

In [4]:
df=pd.read_csv('Pilot_Input_Crop_Cassava.csv') 

# Cleaning data commands

#df=df.drop(df[df['harv_t']==0].index) #Deleting any point that has zero harvesting area
#df=df.reset_index()    #reseting the index after deleting the zero value points
#del df['index']        #The previous step will generate a new column form the old index, this step deletes this column with old index

df.head(5)

,country,admin_1,lon,lat,total_area,harv_area,curr_yield,max_yield,gw_depth,sw_dist,...,tmin_3,tmin_4,tmin_5,tmin_6,tmin_7,tmin_8,tmin_9,tmin_10,tmin_11,tmin_12
0,moz,Zambezia,36.708337,-15.04167,323,200.1,4500,6000,37,92875,...,10.0161,14.8924,19.814301,24.798201,28.101999,27.357500,23.091101,16.896299,9.71018,5.52086
1,moz,Zambezia,36.791670,-15.04167,124,22.4,4500,6000,34,51613,...,10.0161,14.8924,19.814301,24.798201,28.101999,27.357500,23.091101,16.896299,9.71018,5.52086
2,moz,Zambezia,36.875003,-15.04167,195,34.4,4500,6000,29,37562,...,10.0390,14.9651,19.886801,24.865999,28.084400,27.346201,23.111000,16.821600,9.63013,5.43547
3,moz,Zambezia,36.958337,-15.04167,291,28.2,4500,6000,43,33625,...,10.0390,14.9651,19.886801,24.865999,28.084400,27.346201,23.111000,16.821600,9.63013,5.43547
4,moz,Zambezia,37.125003,-15.04167,402,88.7,4500,6000,17,31068,...,10.0761,14.9526,19.824699,24.807501,28.113001,27.369101,23.100300,16.906200,9.71928,5.52977


### Calculating 

* Reference evapotranspiration (ETo)
* Crop coefficient (kc)
* Crop evaporotransiration (ETc)
* Effective rainfall (eff)

### Reference evapotranspiration (ETo)

ETo is estimated based on FAO's **Penman-Monteith** formula. The reference evapotranspiration, ETo, provides a standard to which:

* evapotranspiration at different periods of the year or in other regions can be compared
* evapotranspiration of other crops can be related

**Sources**
- [FAO Irrigation and Drainage Paper No. 56, Chapter 2](http://www.fao.org/3/X0490E/x0490e06.htm) 
- [Andreas P. SAVVA Karen FRENKEN, "Irrigation manual", Volume 1, Module 4](http://www.fao.org/tempref/agl/AGLW/ESPIM/CD-ROM/documents/7I1_e.pdf)
- [Lincoln Zotarelli, Michael D. Dukes, Consuelo C. Romero, Kati W. Migliaccio, and Kelly T.
Morgan, "Step by Step Calculation of the Penman-Monteith
Evapotranspiration (FAO-56 Method)"](http://www.agraria.unirc.it/documentazione/materiale_didattico/1462_2016_412_24509.pdf)
- [Richard G. ALLEN et al., "Crop evapotranspiration - Guidelines for computing crop water
requirements - FAO Irrigation and drainage paper 56"](https://appgeodb.nancy.inra.fr/biljou/pdf/Allen_FAO1998.pdf)


##### TODO list

- [X] Fill in the sources
- [X] Review and validate equations

--------------------------------------------------------------------------------------------------------------

In [5]:
# Defining function
def evap_i(lat,elev,wind,srad,tmin,tmax,tavg,month):
    if month ==1:
        J = 15
    else:
        J = 15 + (month-1)*30
        
    latitude = pyeto.deg2rad(lat)
    atmosphericVapourPressure = pyeto.avp_from_tmin(tmin)
    saturationVapourPressure = pyeto.svp_from_t(tavg)
    ird = pyeto.inv_rel_dist_earth_sun(J)
    solarDeclination = pyeto.sol_dec(J)
    sha = [pyeto.sunset_hour_angle(l, solarDeclination) for l in latitude]
    extraterrestrialRad = [pyeto.et_rad(x, solarDeclination,y,ird) for x, y in zip(latitude,sha)]
    clearSkyRad = pyeto.cs_rad(elev,extraterrestrialRad)
    netInSolRadnet = pyeto.net_in_sol_rad(srad*0.001, albedo=0.23)
    netOutSolRadnet = pyeto.net_out_lw_rad(tmin, tmax, srad*0.001, clearSkyRad, atmosphericVapourPressure)
    netRadiation = pyeto.net_rad(netInSolRadnet,netOutSolRadnet)
    tempKelvin = pyeto.celsius2kelvin(tavg)
    windSpeed2m = wind
    slopeSvp = pyeto.delta_svp(tavg)
    atmPressure = pyeto.atm_pressure(elev)
    psyConstant = pyeto.psy_const(atmPressure)
    
    return pyeto.fao56_penman_monteith(netRadiation, tempKelvin, windSpeed2m, saturationVapourPressure, atmosphericVapourPressure, slopeSvp, psyConstant, shf=0.0)

#Initiate
for i in range(1,13):
    df['ETo_{}'.format(i)]=0  ##To make sure that it is reset to zero

# calculate ETo for each row for each month 
# range(1,13) and .format(i): to generate monthly calculation of ETo
for i in range(1,13):
    df['ETo_{}'.format(i)] = evap_i(df['lat'],df['elevation'],df['wind_{}'.format(i)],df['srad_{}'.format(i)],df['tmin_{}'.format(i)],df['tmax_{}'.format(i)],df['tavg_{}'.format(i)],i)

### Define rainfall pattern (unimodal vs bimodal vs trimodal)

For this example we assume that unimodal pattern for the whole study area, which means it has one raining season only. 

##### TODO list

- [ ] Identify agroecological zones in Mozambique - issue #6
- [ ] Decide on crops we are modelling in the pilot case study - issue #6
- [ ] Specify crop calendar per region and crop - issue #6
- [ ] Modify input files as needed to reflect the above - issue #6

--------------------------------------------------------------------------------------------------------------

In [6]:
df['Mode']=('region_1')

### Calculate kc based on the different growth stages

Note that the user shall define the kc values for different stages of a crop. In this case values of 0.8, 0.9, 1 and 0.8 were used for the 4 growth stages of cassava. Source is available [here](http://www.fao.org/3/X0490E/x0490e0b.htm).

**Other sources**
- [Fatemeh Aghdasi, "Crop Water Requirement Assessment.."](https://webapps.itc.utwente.nl/librarywww/papers_2010/msc/wrem/aghdasi.pdf), 2010
- [FAO Irrigation potential in Africa Chapter 3"](http://www.fao.org/3/S2022E/s2022e07.htm)

##### TODO list

- [X] Merge Li1, Li2 and kci1, kci2 into  - issue #8

--------------------------------------------------------------------------------------------------------------

In [7]:
# Define kc function and its attributes

def kc(plantation,Li,Ld,Lm,Le,kci,kcd,kcm,kce,isodate): 
    
    """
Each crop goes through four growing stages: initial - development - mid-season and end-season (check FAO-56 chapter 6 for more details)

Inputs:
Plantation = plantation datetime 
Li = length of the initial stage (in days)
Ld = length of the development stage (in days)
Lm = length of the mid-season stage (in days)
Le = length of the end-season stage (in days)

kci = crop coefficient 'kc' at the initial stage. In this stage the ckc value is constant and equal to kci
kcm = crop coefficient 'kc' at the mid-season stage.  In this stage the ckc value is constant and equal to kcm
kce = crop coefficient 'kc' at the end-season stage. In this stege the ckc value varies linearly between kce and kcm (check equation 66 - page 132, FAO56). 
isodate = current date (optional)

Outputs: 
* ckc : current crop coefficient, which is constant in the initial and mid-season stages and varies linearly in the development (increasing) and end-season (declining) stages. 

Some Examples:
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-01-20")
        >>> 0.15
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-02-10")
        >>> 0.774
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-03-12")
        >>> 1.19
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-04-06")
        >>> 0.559
    
    """
#step 1: 
    
    plantation = pd.to_datetime(plantation, format='%d/%m') #converting the plantation input info to data time
    isodate = pd.to_datetime(isodate , format='%d/%m')  #converting the current date input info to data time
    test = ((isodate-plantation).days)%365   #The difference in days between the current day and the plantation day.
    
    # Setting the plantation date and the current date (this is not used)
    Jc = test   
    Jp = 0
    J = (Jc - Jp)%365  # %365 means the remaing days of the year
    
#Step 2: Calculating the day of the year when each crop stage ends placing the date in the number of days year betweem 0 (1/jan) and 365 (31/Jan)
    JLi = Jp + Li    #end of initial stage = plantation date + lenght of initial stage
    JLd = JLi + Ld   #end of development stage = end of initial stage + length of development stage
    JLm = JLd + Lm   #end of mid-season stage = end of development stage + length of mid-season stage
    JLe = JLm + Le   #end of end-season stage = end of mid-season stage + length of end-season stage

#step 3: calculating ckc based on the end of each stage date

    if Jc > Jp and Jc < JLe:   #if the current date is greater than the plantation date and it is greater than the end of end-season stage
        if J <= JLi:    
            ckc = kci  #if the current date is before the end of initial stage then ckc = kci the coefficient of the initial stege
        elif Jc > JLi and Jc <=JLd:  #if the current date is betweeen the end of the intial stege and the end of the development stage, then ckc is computed based on equation 66 (page 132.FAO56)
            ckc = kci + ((Jc-JLi)/Ld * (kcm-kci))
        elif Jc > JLd and Jc <= JLm: 
            ckc = kcm
        elif Jc > JLm and Jc <= JLe:
            ckc = kcm + ((Jc-JLm)/Le * (kce-kcm))
            
    else:
        ckc = 0
    
    return ckc


### Running the function

# Import sample crop calendar
mode = pd.read_excel('Pilot_Input_Crop_Calendar.xlsx')

#Note: The code here is adjusted to avoid the end of year issue. In other cases, the init1 and init2 are one stage init:
#pay attention to all changes, you may need to change this if the crop calendar change 

#Planting season: Initial Stage  (plant = init)
init_start = pd.to_datetime(mode['init_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
init_end = pd.to_datetime(mode['init_end'], format='%d/%m')
mode['init_start_month'] = init_start.dt.month
mode['init_end_month'] = init_end.dt.month
mode['init_days'] = abs(init_end - init_start).dt.days #Calculating the length of the planting season
Li = abs(init_end - init_start).dt.days

#growing 1: Development Stage (grow = dev)
dev_start = pd.to_datetime(mode['dev_start'], format='%d/%m')
dev_end = pd.to_datetime(mode['dev_end'], format='%d/%m')
mode['dev_start_month'] = dev_start.dt.month
mode['dev_end_month'] = dev_end.dt.month
mode['dev_days'] = abs(dev_end - dev_start).dt.days
Ld = abs(dev_end - dev_start).dt.days 

#growing 2: Mid stage ( add : mid)
mid_start = pd.to_datetime(mode['mid_start'], format='%d/%m')
mid_end = pd.to_datetime(mode['mid_end'], format='%d/%m')
mode['mid_start_month'] = mid_start.dt.month
mode['mid_end_month'] = mid_end.dt.month
mode['mid_days'] = abs(mid_end - mid_start).dt.days
Lm = abs(mid_end - mid_start).dt.days 

#Harvesting: Late stage (harv = late)
late_start = pd.to_datetime(mode['late_start'], format='%d/%m') #defining the plant start date from excil and setting the correct month and days sequence to read.
late_end = pd.to_datetime(mode['late_end'], format='%d/%m')
mode['late_start_month'] = late_start.dt.month
mode['late_end_month'] = late_end.dt.month
mode['late_days'] = abs(late_end - late_start).dt.days #Calculating the length of the planting season
Le = abs(late_end - late_start).dt.days

for i in range(1,13):
    mode['kc_{}'.format(i)]=0
    
for index,row in mode.iterrows():
    for i in range(0,12):
        init_start = pd.to_datetime(mode['init_start'].iloc[index], format='%d/%m') #read the plant start date from excel. 
        day_start= (init_start.day+1-31)%31   #what does this represent??   
        
        if (init_start.day-1==30):
            month_start = (init_start.month+1-12)%12  #next month
        else:
            month_start = (init_start.month-12)%12  #the current month
            
        month_start = (month_start+i)%12
        if (month_start==0):
            month_start = 12
        mode.loc[index,'kc_{}'.format(month_start)] = kc(mode['init_start'].iloc[index],mode['init_days'].iloc[index],mode['dev_days'].iloc[index],mode['mid_days'].iloc[index],mode['late_days'].iloc[index],0.8,0.9,1,0.8,'{}/{}'.format(day_start,month_start))
        #print (kc)
        
# so far we worked with (df) dataframe which contains GIS outputs, then we created a (mode) dataframe. 
# Here we merge them on into one new dataframe called (data) and we chose the merging to be on the 'Mode' column 

data = pd.merge(df, mode, on='Mode') 

### Calculating crop evapotransiration (ETc)

Note! This is also refered to as Crop Water Requirements (CWR)

**Sources**

See [here](https://www.sciencedirect.com/topics/agricultural-and-biological-sciences/crop-water-requirement) for definitions

In [8]:
# Estimate monthly crop evaropotransoration ETc
for i in range(1,13):
    data['ETc_{}'.format(i)] = data['ETo_{}'.format(i)] * data['kc_{}'.format(i)]

### Sum precipitation

(Not sure if needed; if not it can be deleted)

In [9]:
# Calculating the annual precipitation: which is the sum of precipitation values
data['precipitation_annual']=data.filter(like='prec_').sum(axis=1)  #Filter is used to specify the column of interest

### Calculate effective rainfall for every row for each month

##### TODO list

- [X] Fill in the sources for effective rainfall formulae - issue #1
- [X] Review & Update effective rainfall formula - issue #1
- [ ] Update code so that Water storage capacity of the soil (awsc) changes based on soil type; the latter to come from GIS

--------------------------------------------------------------------------------------------------------------

Effective rainfall calculation is based on [USDA‐SCS method](http://www.fao.org/3/x5560e/x5560e03.htm#TopOfPage), expressed through the emperical formula presented by [M. Ali, S. Mubarak](https://doi.org/10.9734/ARJA/2017/36812). 

Note that usable soil water storage (d) is defined in mm. d depends on soil water holding capacity, root depth as shown [here](http://www.droughtmanagement.info/literature/BC_MA_Soil_Water_Storage_Capacity_2005.pdf). It is generally calculated as 40 to 60 percent of the available soil-water capacity in the crop root zone, depending on the irrigation management practices used. In this analysis we use 50%.


**Sources:**
 - USDA‐SCS Method, Chapter 2 "Irrigation Water Requirements", page 147, url:https://www.wcc.nrcs.usda.gov/ftpref/wntsc/waterMgt/irrigation/NEH15/ch2.pdf
 - FAO, "Effective rainfall in irrigated agriculture", emperical methods, url: http://www.fao.org/3/x5560e/x5560e00.htm#Contents
 - M. Ali, S. Mubarak, "Effective Rainfall Calculation Methods for Field Crops: An Overview, Analysis and New Formulation", url: https://doi.org/10.9734/ARJA/2017/36812
 - FAO, "Irrigation Water Management: Irrigation Water Needs", url: http://www.fao.org/3/S2022E/s2022e00.htm#Contents
 - S. Mohan, B. Simhadrirao, N. Arumugam, "Comparative study of effective rainfall estimation methods for lowland rice", url: https://link.springer.com/article/10.1007/BF00698810
 - Balram Panigrahi, Megh R. Goyal, (Book), "Soil and Water Engineering: Principles and Applications of Modeling", page 265, url: https://books.google.se/books?id=wR9jDAAAQBAJ&pg=PA264&lpg=PA264&dq=criwar+semi+empirical+information+Pe&source=bl&ots=Bim8aJhvm3&sig=ACfU3U3xQK7FeZxIutU1-W962mydu3yesQ&hl=en&sa=X&ved=2ahUKEwjH0e3H-JriAhUxtIsKHftyB_UQ6AEwCnoECAcQAQ#v=onepage&q=criwar%20semi%20empirical%20information%20Pe&f=false


In [10]:
# Effective Rooting Depth of Mature Crops (rd) in m; defined by user. 
# Indicative values available here (http://www.droughtmanagement.info/literature/BC_MA_Soil_Water_Storage_Capacity_2005.pdf) 
data["rd"] = 0.79  # m

# Water storage capacity of the soil (awsc) in mm/m; defined by user.
# Indicative values for different soils available here (http://www.droughtmanagement.info/literature/BC_MA_Soil_Water_Storage_Capacity_2005.pdf)

# Defining usable soil water storage (d) in inches
data["da"] = data["rd"] * data["awsc"] * 0.5 * 0.0393701

# correction factor; depends on da (see Balram Panigrani above)
data["sf"] = 0.531747 + 0.295164*data["da"] - 0.057697*(data["da"]**2) + 0.003804*(data["da"]**3)

#Define rainfall function 
def eff_rainfall(sf, prec, etc):
    return (sf*(0.70917*(((prec*0.0393701) ** 0.82416) - 0.11556))*(10**(0.2426*(etc*0.0393701))))/0.0393701


#Initiate
for i in range(1,13):
    data['eff_{}'.format(i)]=0
    
for i in range(1,13):
    data['eff_{}'.format(i)] = eff_rainfall(data["sf"], data['prec_{}'.format(i)],data['ETc_{}'.format(i)])
    data.loc[data['eff_{}'.format(i)] < 0, 'eff_{}'.format(i)] = 0.0001
    data.loc[(data['eff_{}'.format(i)] >= data['prec_{}'.format(i)]), 'eff_{}'.format(i)] = data['prec_{}'.format(i)]
    data.loc[(data['eff_{}'.format(i)] >= data['ETc_{}'.format(i)]), 'eff_{}'.format(i)] = data['ETc_{}'.format(i)]

## Uncomment to save file up to this point

##Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('Pilot_Result_Part1.xlsx', engine='xlsxwriter')
#
## Convert the dataframe to an XlsxWriter Excel object.
#data.to_excel(writer, sheet_name='part_1')
#
## Close the Pandas Excel writer and output the Excel file.
#writer.save()

## Part 2. Calculating Irrigation requirements 

* Net Irrigation requirements (IRn)
* Peak Crop Water Requirements (PCWR)
* Peak Water Demand (PWD) 
* Seasonal Scheme Water Demand (SSWD)

### Net Irrigation requirements (IRn) (mm/month)

**Sources**

- [FAO paper 24, "Crop Water Requirements"](http://www.fao.org/3/s8376e/s8376e.pdf)
- [FAO, "Irrigation potential in Africa Chapter 5"](http://www.fao.org/3/W4347E/w4347e0c.htm#chapter%205:%20irrigation%20water%20requirements)
- [Andreas P. SAVVA Karen FRENKEN, "Irrigation manual", Volume 1, Module 4](http://www.fao.org/tempref/agl/AGLW/ESPIM/CD-ROM/documents/7I1_e.pdf)


##### TODO list

- [X] Fill in the sources for IRn - issue #9
- [X] Review & Update IRn formula as/if needed - issue #9

--------------------------------------------------------------------------------------------------------------

In [11]:
for i in range (1,13):
    data['IRn_{}'.format(i)]= data['ETc_{}'.format(i)]*30 - data['eff_{}'.format(i)]*30

### Peak Crop Water Requirements (PCWR)

##### TODO list

- [X] Fill in the sources - issue #10
- [X] Review & Update formulas as needed - issue #10

In [12]:
# Converting IRn into (m3/ha per month) 
for i in range (1,13):
    data['IRn_{}'.format(i)] *= 10    # 0.001*10000
    
# Converting IRn into (m3/ha per day)
for i in range (1,13):
    data['IRnd_{}'.format(i)] = data['IRn_{}'.format(i)] / 30
    
# Peak crop water requirement (PCWR) is estimated as 2*IRnd (source: FAO manual) (unit: m3/ha per day)
for i in range (1,13):
    data['PCWR_{}'.format(i)] = data['IRnd_{}'.format(i)] * 2 
    
# Converting PCWR into  l/s/ha "Duty"
for i in range (1,13):
    data['PCWR_{}'.format(i)] *= 0.012

### Peak Water Demand (PWD)  in l/s || Seasonal Scheme Water Demand (SSWD) in m3

##### TODO list

- [ ] Validate assumptions on efficiencies & operating hours
- [ ] Update the 'harve_i_ha' column when right input data are available

In [13]:
# In order to estimate PWD and SSWS we need first to compute the irrigated area used in that particular month. 

for index,row in data.iterrows():
    len_init = (len(range(row['init_start_month'],row['init_end_month']))+1)
    
    
# PWD = PCWR / Irrigation efficiency(IrrEff) 
# IrrEff = Field Application Efficiency (aeff) * Distribution Efficiency (deff)*100 
# deff = (Conveyance efficiency + field canal efficiency)
# deff: 0.95 (all scenarios)
# aeff: 0.6 (Surface Irr), 0.75 (Sprinkler Irr), 0.9 (Drip Irr)

pumping_hours_per_day=8    # Assumption  
deff= 0.95                 # Assumption
aeff= 0.75                 # Assumption


count_p=0                           # To adjust the count of months in the loop below
count_h=0                           # To adjust the count of months in the loop below
init_count = np.zeros(len(data))
late_count = np.zeros(len(data))

for i in [1,2,3,4,5,6,7,8,9,10,11,12]:

    init = [(i >= j) & (i <= k) for j, k in zip(data['init_start_month'],data['init_end_month'])]

    data.loc[init,'harvested_{}'.format(i)] =(data['harv_area']/(len_init)*init_count)
    data.loc[init,'PWD_{}'.format(i)]= (data['PCWR_{}'.format(i)] *(data['harvested_{}'.format(i)]*24))/(pumping_hours_per_day*aeff*deff)
    data.loc[init,'SSWD_{}'.format(i)]= (data['IRn_{}'.format(i)]*(data['harvested_{}'.format(i)])/(aeff*deff))
    
    dev = [(i >= j) & (i <= k) for j, k in zip(data['dev_start_month'],data['dev_end_month'])]
    
    data.loc[dev,'harvested_{}'.format(i)]=data['harv_area']
    data.loc[dev,'PWD_{}'.format(i)]=(data['PCWR_{}'.format(i)]*data['harv_area']*24)/(pumping_hours_per_day*aeff*deff)
    data.loc[dev,'SSWD_{}'.format(i)]= (data['IRn_{}'.format(i)]*data['harv_area'])/(aeff*deff)
    
    
    mid = [(i >= j) & (i <= k) for j, k in zip(data['mid_start_month'],data['mid_end_month'])]
    
    data.loc[mid,'harvested_{}'.format(i)]=data['harv_area']
    data.loc[mid,'PWD_{}'.format(i)]=(data['PCWR_{}'.format(i)]*data['harv_area']*24)/(pumping_hours_per_day*aeff*deff)
    data.loc[mid,'SSWD_{}'.format(i)]= (data['IRn_{}'.format(i)]*data['harv_area'])/(aeff*deff)
    
    late = [(i >= j) & (i <= k) for j, k in zip(data['late_start_month'],data['late_end_month'])]

    late_count += late * 1

    data.loc[late,'harvested_{}'.format(i)]=(data['harv_area']/([len(range(i,j+1)) for i,j in zip(data['late_start_month'],data['late_end_month'])])*late_count)
    data.loc[late,'PWD_{}'.format(i)]= (data['PCWR_{}'.format(i)]*(data['harvested_{}'.format(i)]*24)/(pumping_hours_per_day*aeff*deff))
    data.loc[late,'SSWD_{}'.format(i)]= (data['IRn_{}'.format(i)]*(data['harvested_{}'.format(i)])/(aeff*deff))

## Export dataframe into a csv file

(Uncomment to activate)

In [14]:
## Finally, print results of part 2
#
##Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('Pilot_Results.xlsx', engine='xlsxwriter')
#
## Convert the dataframe to an XlsxWriter Excel object.
#data.to_excel(writer, sheet_name='test_all')
#
#
## Close the Pandas Excel writer and output the Excel file.
#writer.save()

## Part 3.  Estimatind energy and power requirements

***Sources***

- [FAO, Chapter 4 - *Comparison of energy alternatives for small-scale irrigation"](http://www.fao.org/3/u2246e/u2246e05.htm#1%20technical%20calculations)
- [Andreas P. SAVVA Karen FRENKEN, "Irrigation manual", Volume 1, Module 4](http://www.fao.org/tempref/agl/AGLW/ESPIM/CD-ROM/documents/7I1_e.pdf)


##### TODO list

- [ ] Fill in other sources - issue #20
- [ ] Review & Update formula as/if needed - issue #20
- [ ] Cross check efficiencies for double calculating - issue #20

### Total dynamic head (TDH) for ground and surface water sources

In [15]:
#data['sw_depth'] = np.random.randint(0,5, size=len(data))  ## refering to the suction lift im (m)
#data['sw_dist'] = np.random.randint(0,50, size=len(data))  ## refering to the distance to water source in (m)
#data['sw_suitability'] = np.random.choice([1,9999], size=len(data))  ## refering to the distance to water source in (m)

# Total dynamic head (TDH) in meters: Total static head (m) + Pressure head (m) + Friction head (m)
# Total static head for groundwater: ground water depth gw_depth
# Total static head for surface water: (elevation of water source - elevation of application) x distance to source

# Pressure head (m): 0 m (SU), 30 m (SP), 10 m (DR) , check if it is in 1992?
# Friction head (m): 1 m (SU), 20% of presure head (SP, DR) , check if it is in 1992?

pres_head_gw = 1
frict_head_gw = 0.2*pres_head_gw

pres_head_sw = 1
frict_head_sw = 0.2*pres_head_sw

def tdh_gw(row):
    tdh=(row['gw_depth']) + pres_head_gw + frict_head_gw
    return tdh

def tdh_sw(row):
    tdh=(row['sw_depth']*row["sw_dist"]*row["sw_suit_idx"]) + pres_head_sw + frict_head_sw  
    return tdh

data['tdh_gw'] = data.apply(tdh_gw , axis=1)
data['tdh_sw'] = data.apply(tdh_sw , axis=1)  

### Estimating power (kW) and electricity (kWh) demand 

In [16]:
#Setting the default value for these parameters
for i in range (1,13):
    data['PD_E_gw_{}'.format(i)]=0      #PD_E_gw: Peak Demand (kw) using electric powered pump for ground water
    data['PD_E_sw_{}'.format(i)]=0      #PD_E_sw: Peak Demand (kw) using electric powered pump for surface water 
    data['ED_E_gw_{}'.format(i)]=0      #ED_E_gw: Electricity Demand (kwh) using electric powered pump for ground water
    data['ED_E_sw_{}'.format(i)]=0      #ED_E_sw: Electricity Demand (kwh) using electric powered pump for surface water 

# Pumping plant efficiencty (%)= fuel efficiency (%) * "power unit eff (%)" * transmission eff (%) * pump eff (%) * 100%
# The Power Unit: can be diesel engine or electric engine. In the first we call it (diesel powered pump) and the second (electric powered pump)

# Diesel powered pump
#Worst case: 0.9*0.3*0.9*0.4 ~ 10 % (0.1)
#Best case: 1*0.4*1*0.8 = 32% (0.32)

# Electric powered pump
#Worst case: 0.9*0.75*0.9*0.4 ~ 25% (0.25)
#Best case: 1*0.85*1*0.8 ~ 70% (0.7)

pump_plant_eff=0.7

for i in range (1,13):
    PWD = 'PWD_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    PD_E_gw = 'PD_E_gw_{}'.format(i)
    ED_E_gw = 'ED_E_gw_{}'.format(i)
    PD_E_sw = 'PD_E_sw_{}'.format(i)
    ED_E_sw = 'ED_E_sw_{}'.format(i)

    data[PD_E_gw]=(9.81*(data[PWD]/1000)*data['tdh_gw'])/pump_plant_eff
    data[ED_E_gw]=(data[SSWD]*data['tdh_gw']*0.00272)/pump_plant_eff
    data[PD_E_sw]=(9.81*(data[PWD]/1000)*data['tdh_sw'])/pump_plant_eff
    data[ED_E_sw]=(data[SSWD]*data['tdh_sw']*0.00272)/pump_plant_eff

### Decision between ground or surface water irrigation

Selection based on peak power demand (kW) for the two options. First, we identify the max value between the two sub-categories and then we select the option that provides the minimum peak power. 

In [17]:
data['PD_E_gw_max']=data.filter(like='PD_E_gw_').max(axis=1) 
data['PD_E_sw_max']=data.filter(like='PD_E_sw_').max(axis=1)  
data['PD_E'] = np.minimum.reduce(data[['PD_E_gw_max', 'PD_E_sw_max']].values, axis=1)

### Estimate annual electricity demand (kWh/year) 

**Sources**

- [Andreas P. SAVVA Karen FRENKEN, "Irrigation manual", Volume 1, Module 5](http://www.fao.org/tempref/agl/AGLW/ESPIM/CD-ROM/documents/7I1_e.pdf)

In [18]:
# Estimate the gross annual irrigation requirements per location in (m3)
data['gross_an_irrig_req']=data.filter(like='SSWD_').sum(axis=1)   

# Identify the peak water demand (PWD) and convert it from l/s to m3/h
data['PWD_max']=data.filter(like='PWD_').max(axis=1)
data['PWD_max'] *= 3.6

# Estimate annual electricity demand per location assuming motor efficiency
motor_eff = 0.88
data['Annual_elec_demand'] = (data['gross_an_irrig_req']/data['PWD_max'])*(data['PD_E']/0.88)

In [19]:
# Finally, print results

#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Pilot_Results_Final.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
data.to_excel(writer, sheet_name='test_all')


# Close the Pandas Excel writer and output the Excel file.
writer.save()